# Point clouds

Point clouds are a specific type of file: they contain a set of points, located in a 3d space (sometimes with more optional arguments). They are very popular to store LiDAR data and can be used to produce digital elevation models (DEMs). For this, we will be using the `laspy` python library. Optionally, you can also download [cloud compare](https://www.danielgm.net/cc/) to visualize point clouds. Similarly to the [raster formats tutorial](./raster_formats.ipynb), we will start by presenting classic `.las` files and then their cloud-optimized versions (COPC: Cloud Optimized Point Cloud).

## Basic usage of .las files

First let's import the required libraries

In [38]:
import laspy
import numpy as np
import pandas as pd

from utils import get_file_size_in_mb

You will need a `.las` file ; in this notebook, a sample file from the `laspy` example will be used (downloadable [here](https://github.com/laspy/laspy/blob/master/tests/data/simple.las)). For detailed informations on `.las` file see [here](https://laspy.readthedocs.io/en/latest/intro.html). Here is how to open a `.las` file:

In [39]:
las_file =  "./sample_data/point_clouds/simple.las"

with laspy.open(las_file) as fh:
    las = fh.read()
    n_points = fh.header.point_count
    # number of points
    print(f"The file contains {n_points} points")
    # dimensions
    print("\nFile dimensions:")
    for dimension in las.point_format.dimensions:
        print(dimension.name)

The file contains 1065 points

File dimensions:
X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
red
green
blue


We can also visualize this point cloud in an interactive plot with hvplot, using the intensity as a color indicator:

In [40]:
import holoviews as hv
from holoviews import dim, opts
hv.extension('plotly')

with laspy.open(las_file) as fh:
    las = fh.read()
    x, y, z = las.X, las.Y, las.Z
    intensity = las.intensity

data = pd.DataFrame({'x': x, 'y': y, 'z': z, 'intensity': intensity})
hv.Scatter3D((data), kdims=['x', 'y', 'z']).opts(cmap='plasma', color="intensity", size=1.5, colorbar=True, height=600, width=600)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

:Scatter3D   [x,y,z]   (intensity)

All the raw data is stored as numpy arrays, meaning we can use numpy methods. For example, we can easily filter the data we're interested in: let's say we want to remove all low-intensity (intensity < 10):

In [4]:
low_threshold = 50
high_threshold = 150

# create the mask based on the intensity
intensity = las.intensity
intensities_mask = (intensity > low_threshold) & (intensity < high_threshold)
masked_out_values = len(intensity) - np.sum(intensities_mask)
print(f"{masked_out_values} values are masked out")

# filter out the values
intensity_masked = las.intensity[intensities_mask]
x, y, z = las.X, las.Y, las.Z
x_masked = x[intensities_mask]
y_masked = y[intensities_mask]
z_masked = z[intensities_mask]

data = pd.DataFrame({'x': x_masked, 'y': y_masked, 'z': z_masked, 'intensity': intensity_masked})
hv.Scatter3D((data), kdims=['x', 'y', 'z']).opts(cmap='plasma', color="intensity", size=1.5, colorbar=True, height=600, width=600)

648 values are masked out


:Scatter3D   [x,y,z]   (intensity)

The low intensity points were successfully filtered out.


Another important feature of `laspy` is adding new dimensions. For example, let's define a `brightness` dimension which is the norm of the red, green and blue values:

$$B=\sqrt{{red}^2 + {green}^2 + {blue}^2}$$

Here is how to compute it, add it to the file and use it in the plot instead of the intensity

In [5]:
with laspy.open(las_file) as fh:
    las = fh.read()

r, g, b = las.red, las.green, las.blue

brightness = np.linalg.norm([r, g, b], axis=0).astype(np.uint32)

las.add_extra_dim(laspy.ExtraBytesParams(
    name="brightness",
    type=np.uint32,
    description="Norm of rgb values"
))

las.brightness = brightness

for dimension in las.point_format.dimensions:
    print(dimension.name)


data = pd.DataFrame({'x': x, 'y': y, 'z': z, 'brightness': brightness})
hv.Scatter3D((data), kdims=['x', 'y', 'z']).opts(cmap='plasma', color="brightness", size=1.5, colorbar=True, height=600, width=600)

X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
red
green
blue
brightness


:Scatter3D   [x,y,z]   (brightness)

Finally, we can write this data into a new file:

In [6]:
output_file_path = "./sample_data/point_clouds/output_file.las"
las.write(output_file_path)

## Dealing with large files

Sometimes, files can get too large to be fully loaded in RAM. In this case, the same strategy as for any data type is used: read/write and process the file by chunk. This is easily implemented in `laspy`. In this example, we want to 

1. Read an input file by chunk
2. Create a new file, which will only contain the filtered intensity
3. Write this output file by chunk

In [7]:
las_file = "./sample_data/point_clouds/simple.las"

with laspy.open(las_file) as f:
    for points in f.chunk_iterator(300):  # only read 300 points everytime (max)
        print(len(points))

300
300
300
165


Here is a more complete example of reading a file, applying a filter to its intensity values and writing it, chunk by chunk:

In [8]:
input_file = "./sample_data/point_clouds/simple.las"
output_file = "./sample_data/point_clouds/filtered_by_chunk.las"
intensity_threshold = 50
chunk_size = 300

with laspy.open(input_file) as f:
    with laspy.open(output_file, mode="w", header=f.header) as writer:
        for points in f.chunk_iterator(chunk_size):
            filtered_points = points[points.intensity > intensity_threshold]  # filter usefull points
            writer.write_points(filtered_points)

This greatly improves reading and writing point cloud files, but we can also easily optimize their disk space usage. For this, we save the files as `.laz` instead: laz is a lossless compression format for point clouds. Make sure you also installed the required backend for `.laz` files (https://laspy.readthedocs.io/en/latest/installation.html#pip).

In [9]:
input_file = "./sample_data/point_clouds/simple.las"
output_file = "./sample_data/point_clouds/simple_compressed.laz"

with laspy.open(input_file) as fh:
    las = fh.read()
las.write(output_file)

las_file_size = get_file_size_in_mb(input_file)
laz_file_size = get_file_size_in_mb(output_file)

print(f".las size: {las_file_size} MB")
print(f".laz size: {laz_file_size} MB")

.las size: 0.036437 MB
.laz size: 0.018217 MB


This is an easy way to compress point cloud files.

## Cloud optimized point clouds (COPC)

As with any data format, new cloud-optimized formats appeared. As explained in the introduction, COPC isn't a different file format (i.e. files are still `.las` and `.laz`), rather a different way of organizing data in the file. For more details, check out the [official COPC website](https://copc.io/). In order to use COPCs with `laspy`, make sure you have installed the `lzrs` dependency (also used for dealing with `.laz` files, see the [installation documentation](https://laspy.readthedocs.io/en/latest/installation.html#cloud-optimized-point-cloud-copc)). Example files for this section were taken from https://copc.io/#example-data. Although it is possible to open local COPC files (like [this one](https://github.com/PDAL/data/blob/master/autzen/autzen-classified.copc.laz)), it can be more intersting to open a file from a cloud, as this is generally how to deal with COPC files. This script first opens the file without reading all of its data. Using information from the header, it then selects a fraction of the data to read. 

In [10]:
from laspy import Bounds, CopcReader

copc_file_url = "https://hobu-lidar.s3.amazonaws.com/sofi.copc.laz"

with CopcReader.open(copc_file_url) as crdr:
    header = crdr.header
    total_points_count = header.point_count
    print(f"The file contains {total_points_count:,} total points")

    min_coordinates = header.mins  # numpy array: [x_min_coord, y_min_coord, z_min_coord]
    max_coordinates = header.maxs  # numpy array: [x_max_coord, y_max_coord, z_max_coord]
    sizes = header.maxs - header.mins  # numpy array: [x_size, y_size, z_size]
    
    # select a "column" in the data (approx. 25% of the data)
    # x_coords: [xmin, xmin + size_x/2]
    # y_coords: [ymin, ymin + size_y/2]
    # z_coords: [zmin, zmin + size_z]
    x_max = min_coordinates[0] + sizes[0] / 2
    y_max = min_coordinates[1] + sizes[1] / 2
    z_max = min_coordinates[2] + sizes[2]
    selection = np.array([x_max, y_max, z_max])
    query_bounds = Bounds(mins=min_coordinates, maxs=selection)

    # query only the selected data
    points = crdr.query(query_bounds)
    selected_points_count = len(points)
    selected_points_ratio = selected_points_count / total_points_count * 100
    print(f"{selected_points_count:,} points were selected, i.e. approximately {selected_points_ratio:.1f}%")

The file contains 364,384,576 total points
92,459,151 points were selected, i.e. approximately 25.4%


Querying a limited amount of points is primordial to optimize ressources usage. This is also a better way of going through the data than using chunks: this allows you to select what points you want to select. 

## Point cloud with NetCDF

The NetCDF format can also be used to store and manage point cloud data. In this example, the NetCDF point cloud data has been downloaded from the [Hydroweb-next website](https://hydroweb.next.theia-land.fr/). This data comes from the "SWOT Level-2 HR Pixel Cloud Vector" product and contains detailed information about water mask pixels.

The same dataset can also be downloaded using the provided [Python script](./sample_data/point_clouds/download_script.py).

In [42]:
import netCDF4

The dataset can be read with the corresponding library as follows : 

In [49]:
dataset = netCDF4.Dataset('./sample_data/point_clouds/swot_nc.nc')

# Step 2: Extract data variables
latitude = dataset.variables["latitude_vectorproc"][:]
longitude = dataset.variables["longitude_vectorproc"][:]
height = dataset.variables["height_vectorproc"][:]

We can display the number of points and the dimensions of the dataset.

In [44]:
print(f"Number of points  : {len(dataset.dimensions['points'])}\n")

print("Variables :")  
for var_name, var in dataset.variables.items():
    print(f"- {var_name}: dimensions={var.dimensions}, dtype={var.dtype}")


Number of points  : 7763966

Variables :
- azimuth_index: dimensions=('points',), dtype=int32
- range_index: dimensions=('points',), dtype=int32
- latitude_vectorproc: dimensions=('points',), dtype=float64
- longitude_vectorproc: dimensions=('points',), dtype=float64
- height_vectorproc: dimensions=('points',), dtype=float32
- reach_id: dimensions=('points', 'nchar_reach_id'), dtype=|S1
- node_id: dimensions=('points', 'nchar_node_id'), dtype=|S1
- lake_id: dimensions=('points', 'nchar_lake_id'), dtype=|S1
- obs_id: dimensions=('points', 'nchar_obs_id'), dtype=|S1
- ice_clim_f: dimensions=('points',), dtype=int8
- ice_dyn_f: dimensions=('points',), dtype=int8


To visualize this point cloud with hvplot, we use the height as a color indicator. Beforehand, we remove nodata points and we extract a sample of the dataset as it contains too many points to be plotted efficiently.

In [68]:
data = pd.DataFrame({
    "Longitude": longitude,
    "Latitude": latitude,
    "Height": height
})

data = data.dropna()[(data != 0).any(axis=1)] # Remove Nan data and (0,0,0) points

data = data.sample(frac=0.02, random_state=42) #Extract a sample from the data

/tmp/ipykernel_8115/2473443890.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [60]:
hv.Scatter3D((data), kdims=['Latitude', 'Longitude', 'Height']).opts(cmap='plasma', color="Height", size=1.5, colorbar=True, height=600, width=600)

:Scatter3D   [Latitude,Longitude,Height]

We now filter out points with heights less than 44 or greater than 50.

In [72]:
data_filtered = data[(data["Height"] > 44) & (data["Height"] < 50)]

hv.Scatter3D((data_filtered), kdims=['Longitude', 'Latitude', 'Height']).opts(cmap='plasma', color="Height", size=1.5, colorbar=True, height=600, width=600)

:Scatter3D   [Longitude,Latitude,Height]

## Conclusion

Point clouds are an invaluable data format for precisely representing 3D information. They can be compressed and optimized for cloud platforms, and are easily manipulated using `laspy`. `hvplot` can be used to create simple and customizable interactive 3D plots to visualize these points. For more detailed examples, check out the [laspy documentation](https://laspy.readthedocs.io/en/latest/examples.html).